<a href="https://colab.research.google.com/github/Adith-blusim/BP-/blob/main/BPmodel2CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Minute-wise data labelling**

In [ ]:
df = pd.read_csv('mat_15.csv')
df['Timestamp'] = pd.to_datetime(df['Timestamp'])  # Convert timestamp to datetime format
df.set_index('Timestamp', inplace=True)
# Resample the data to count occurrences per minute
count_per_minute = df.resample('T').size()
count_per_minute

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("/content/drive/MyDrive/blusim/roi_updated_data.csv")
df = df.dropna()
df = df.reset_index(drop=True)
df = df.drop(['Unix Timestamp','RESP','ECG','SPO2','PR'],axis=1)

def extract_nibp_values(value):
    # Remove spaces and extra brackets, extract the digits
    cleaned_value = ''.join(filter(str.isdigit, value.replace(' ', '').replace('(', '').replace(')', '')))
    sys_value = cleaned_value[:3]
    dias_value = cleaned_value[3:5]
    return sys_value, dias_value

# Apply the function to "NIBP" column and create new columns "sys" and "dias"
df['sys'], df['dias'] = zip(*df['NIBP'].apply(extract_nibp_values))

rdf = pd.read_csv("/content/drive/MyDrive/blusim/mat_15.csv")
rdf['Timestamp'] = pd.to_datetime(rdf['Timestamp'], format='%H:%M:%S')
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%H:%M:%S')
rdf['Timestamp'] = rdf['Timestamp'].dt.strftime('%H:%M')

df.set_index('Timestamp', inplace=True)
rdf = rdf.groupby('Timestamp').agg(lambda x: x.tolist()).reset_index()
rdf.set_index('Timestamp', inplace=True)
df=df.resample('T').ffill()
df.index = df.index.strftime('%H:%M')

ldf = pd.merge(rdf,df, left_index=True, right_index=True)
ldf[['sys', 'dias']] = ldf[['sys', 'dias']].replace('', np.nan)
ldf.dropna(how='any', inplace=True)
ldf.drop("16:14", inplace=True)
ldf

In [ ]:
#Deleting all values of "sys" greater than 121:
#seconds data

import pandas as pd
import numpy as np
df = pd.read_csv("/content/drive/MyDrive/blusim/roi_updated_data.csv")
df = df.dropna()
df = df.reset_index(drop=True)
df = df.drop(['Unix Timestamp','RESP','ECG','SPO2','PR'],axis=1)

def extract_nibp_values(value):
    # Remove spaces and extra brackets, extract the digits
    cleaned_value = ''.join(filter(str.isdigit, value.replace(' ', '').replace('(', '').replace(')', '')))
    sys_value = cleaned_value[:3]
    dias_value = cleaned_value[3:5]
    return sys_value, dias_value

# Apply the function to "NIBP" column and create new columns "sys" and "dias"
df['sys'], df['dias'] = zip(*df['NIBP'].apply(extract_nibp_values))

rdf = pd.read_csv("/content/drive/MyDrive/blusim/mat_15.csv")
rdf['Timestamp'] = pd.to_datetime(rdf['Timestamp'], format='%H:%M:%S')
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%H:%M:%S')
rdf['Timestamp'] = rdf['Timestamp'].dt.strftime('%H:%M:%S')

df.set_index('Timestamp', inplace=True)
rdf = rdf.groupby('Timestamp').agg(lambda x: x.tolist()).reset_index()
rdf.set_index('Timestamp', inplace=True)
df=df.resample('S').ffill()
df.index = df.index.strftime('%H:%M:%S')

ldf = pd.merge(rdf,df, left_index=True, right_index=True)
ldf[['sys', 'dias']] = ldf[['sys', 'dias']].replace('', np.nan)
ldf.dropna(how='any', inplace=True)
ldf['sys'] = pd.to_numeric(ldf['sys'], errors='coerce')
ldf = ldf.drop(ldf[ldf['sys'].gt(121)].index)
ldf

PCHIP

In [ ]:
from scipy.interpolate import PchipInterpolator

def interpolate_to_120(arr):

    pchip_interpolator = PchipInterpolator(np.arange(len(arr)), arr)
    x_interp = np.linspace(0, len(arr) - 1, 120)
    y_interp = pchip_interpolator(x_interp)
    return y_interp.tolist()

%timeit df_interpolated = ldf[['Raw_Value_ch1','Raw_Value_ch2','Raw_Value_ch3','Raw_Value_ch4']].applymap(interpolate_to_120)
df_interpolated

Cubic Spline

In [ ]:
from scipy.interpolate import CubicSpline
def interpolate_to_120(arr):

    cs_interpolator = CubicSpline(np.arange(len(arr)), arr, bc_type='clamped')
    x_interp = np.linspace(0, len(arr) - 1, 120)
    y_interp = cs_interpolator(x_interp)
    return y_interp.tolist()
%timeit df_interpolated = ldf[['Raw_Value_ch1','Raw_Value_ch2','Raw_Value_ch3','Raw_Value_ch4']].applymap(interpolate_to_120)
df_interpolated

In [ ]:
from scipy.interpolate import CubicSpline
def interpolate_to_120(arr):

    x_interp = np.linspace(0, len(arr) - 1, 120)
    y_interp = np.interp(x_interp, np.arange(len(arr)), arr)
    return y_interp.tolist()
%timeit df_interpolated = ldf[['Raw_Value_ch1','Raw_Value_ch2','Raw_Value_ch3','Raw_Value_ch4']].applymap(interpolate_to_120)
df_interpolated

In [ ]:
len(df_interpolated['Raw_Value_ch2'][45])

**Signal Processing Part**

Butterworth

In [ ]:
!pip install EMD-signal
from PyEMD import EMD
from PyEMD import EEMD
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter

# Sample rate and desired cutoff frequencies (in Hz)
fs = 138.0
lowcut = 0.5
highcut = 6.0

def butter_bandpass(lowcut, highcut, fs, order=3):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=3):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

# Channel_1 = ldf['Raw_Value_ch1']
# Channel_2 = ldf['Raw_Value_ch2']
# Channel_3 = ldf['Raw_Value_ch3']
# Channel_4 = ldf['Raw_Value_ch4']

# Apply the Butterworth band-pass filter to the data
bdf = pd.DataFrame()
bdf['buttered_channel1'] = [butter_bandpass_filter(data, lowcut, highcut, fs, order=3) for data in ldf['Raw_Value_ch1']]
bdf['buttered_channel2'] = [butter_bandpass_filter(data, lowcut, highcut, fs, order=3) for data in ldf['Raw_Value_ch2']]
bdf['buttered_channel3'] = [butter_bandpass_filter(data, lowcut, highcut, fs, order=3) for data in ldf['Raw_Value_ch3']]
bdf['buttered_channel4'] = [butter_bandpass_filter(data, lowcut, highcut, fs, order=3) for data in ldf['Raw_Value_ch4']]

# print(len(buttered_channel2[1]))
bdf

EMD

In [ ]:
eemd = EMD()
imf1c1 = []
imf1c2 = []
imf1c3 = []
imf1c4 = []

for i in bdf['buttered_channel1']:
    time = np.arange(0, len(i)) / fs
    IMFs = eemd.emd(i, time)
    i1 = IMFs[0]
    imf1c1.append(i1)

for i in bdf['buttered_channel2']:
    time = np.arange(0, len(i)) / fs
    IMFs = eemd.emd(i, time)
    i1 = IMFs[0]
    imf1c2.append(i1)

for i in bdf['buttered_channel3']:
    time = np.arange(0, len(i)) / fs
    IMFs = eemd.emd(i, time)
    i1 = IMFs[0]
    imf1c3.append(i1)

for i in bdf['buttered_channel4']:
    time = np.arange(0, len(i)) / fs
    IMFs = eemd.emd(i, time)
    i1 = IMFs[0]
    imf1c4.append(i1)

# print(len(imf1c4[1]))
edf = pd.DataFrame()
edf['IMF1_channel1'] = imf1c1
edf['IMF1_channel2'] = imf1c2
edf['IMF1_channel3'] = imf1c3
edf['IMF1_channel4'] = imf1c4
edf

Hilbert Transform

In [ ]:
from scipy.signal import hilbert
ins_phase_c1 = []
ins_phase_c2 = []
ins_phase_c3 = []
ins_phase_c4 = []

for i in imf1c1:
    analytic_signal = hilbert(i)
    instantaneous_phase = np.angle(analytic_signal)
    ins_phase_c1.append(instantaneous_phase)

for i in imf1c2:
    analytic_signal = hilbert(i)
    instantaneous_phase = np.angle(analytic_signal)
    ins_phase_c2.append(instantaneous_phase)

for i in imf1c3:
    analytic_signal = hilbert(i)
    instantaneous_phase = np.angle(analytic_signal)
    ins_phase_c3.append(instantaneous_phase)

for i in imf1c4:
    analytic_signal = hilbert(i)
    instantaneous_phase = np.angle(analytic_signal)
    ins_phase_c4.append(instantaneous_phase)

hdf = pd.DataFrame()
hdf['instphase_channel1'] = ins_phase_c1
hdf['instphase_channel2'] = ins_phase_c2
hdf['instphase_channel3'] = ins_phase_c3
hdf['instphase_channel4'] = ins_phase_c4
hdf

NN Model Input Design

In [ ]:
mn = 5000
for x in ins_phase_c2:
    if len(x)<mn:
      mn=len(x)
print(mn)
# max(len(ins_phase_c2))
# max(len(hdf['instphase_channel1']))

# standarizing array-size in each cell to be the same as the size of the smallest array
minlen = (min(len(x) for x in ins_phase_c1))
ipc1 = [lst[:minlen] for lst in ins_phase_c1 if len(lst) >= minlen]
ipc2 = [lst[:minlen] for lst in ins_phase_c2 if len(lst) >= minlen]
ipc3 = [lst[:minlen] for lst in ins_phase_c3 if len(lst) >= minlen]
ipc4 = [lst[:minlen] for lst in ins_phase_c4 if len(lst) >= minlen]
# hdf = pd.DataFrame()
# hdf['instphase_channel1'] = ipc1
# hdf['instphase_channel2'] = ipc2
# hdf['instphase_channel3'] = ipc3
# hdf['instphase_channel4'] = ipc4
print(max(len(x) for x in hdf['instphase_channel2']))
print(max(len(x) for x in ipc1))

In [ ]:
merged_ipc = np.stack((ipc1,ipc2,ipc3,ipc4), axis=1)
Y_train = ldf[['sys', 'dias']].to_numpy().astype('float32')
merged_ipc = np.swapaxes(merged_ipc, 1, 2)
merged_ipc.shape

# Y_train.shape
# merged_ipc.dtype

In [ ]:
# a = np.load('merged_ipc.npy')
# if a.all() == merged_ipc.all():
#   print("YEs")
import tensorflow as tf
tf.keras.backend.clear_session()

In [ ]:
from sklearn.model_selection import train_test_split
X_train = merged_ipc
Y_train = ldf[['sys']].to_numpy().astype('float32')
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, Flatten, Dense, BatchNormalization

input_shape =  (64, 4)
model = Sequential()

#first Conv layer
model.add(Conv1D(filters=100, kernel_size=21, strides=1, input_shape=input_shape))
model.add(BatchNormalization())
model.add(tf.keras.layers.ReLU())
model.add(MaxPooling1D(pool_size=2))

#second Conv layer
model.add(Conv1D(filters=200, kernel_size=5))
model.add(BatchNormalization())
model.add(tf.keras.layers.ReLU())
model.add(MaxPooling1D(pool_size=2))

#third Conv layer
model.add(Conv1D(filters=300, kernel_size=5))
model.add(BatchNormalization())
model.add(tf.keras.layers.ReLU())
model.add(MaxPooling1D(pool_size=2))

#GlobalAveragePooling1D layer
model.add(GlobalAveragePooling1D())
#Final dense layer
model.add(Dense(1, activation='linear'))

model.compile(loss=tf.keras.losses.mean_squared_error, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))
model.summary()
history = model.fit(X_train, Y_train, epochs=50, validation_data=(X_test,Y_test), batch_size=64)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# Plotting training and validation loss
plt.figure(figsize=(10, 6))

# Training loss
plt.plot(history.history['loss'], label='Training Loss', marker='o')

# Validation loss
plt.plot(history.history['val_loss'], label='Validation Loss', marker='o')

# Adding labels and title
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Performance for input shape (1650,64,4), no interpolation')
plt.legend()

# Adding grid for better readability
plt.grid(True)

# Show plot
plt.show()

In [ ]:
# y_prediction = model.predict(X_test)
# y_prediction = np.argmax (y_prediction, axis = 1)
# y_test= np.argmax(Y_test, axis=1)
# y_prediction

predictions = model.predict(X_test)
Y_pred = []
for i in range(len(X_test)):
  Y_pred.append(predictions[i])
yf = pd.DataFrame(Y_pred, columns=['predicted'])
yf['test-set'] = Y_test
yf
# Print the first few predictions and corresponding inputs
# for i in range(min(5, len(X_test))):
#     print(f"Input: {X_test[i]}")
#     print(f"Predicted Output: {predictions[i]}")
#     print("\n")

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
y_pred = np.concatenate(Y_pred).ravel()
mae = mean_absolute_error(Y_test, y_pred)
mse = mean_squared_error(Y_test, y_pred)

print(f"Mean Absolute Error: {mae:.2f}")
print(f"Mean Squared Error: {mse:.2f}")

# Plot true vs predicted values
plt.scatter(Y_test, y_pred)
plt.xlabel("True Values")
plt.ylabel("Predicted Values")
plt.title("True vs Predicted Values")
plt.show()

In [ ]:
tf.keras.backend.clear_session()
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, Flatten, Dense, BatchNormalization


X_train = merged_ipc
Y_train2 = ldf[['dias']].to_numpy().astype('float32')
X_train, X_test, Y_train, Y_test2 = train_test_split(X_train, Y_train2, test_size=0.2, random_state=42)

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, Flatten, Dense, BatchNormalization

input_shape =  (64, 4)
model2 = Sequential()

#first Conv layer
model2.add(Conv1D(filters=100, kernel_size=21, strides=1, input_shape=input_shape))
model2.add(BatchNormalization())
model2.add(tf.keras.layers.ReLU())
model2.add(MaxPooling1D(pool_size=2))

#second Conv layer
model2.add(Conv1D(filters=200, kernel_size=5))
model2.add(BatchNormalization())
model2.add(tf.keras.layers.ReLU())
model2.add(MaxPooling1D(pool_size=2))

#third Conv layer
model2.add(Conv1D(filters=300, kernel_size=5))
model2.add(BatchNormalization())
model2.add(tf.keras.layers.ReLU())
model2.add(MaxPooling1D(pool_size=2))

#GlobalAveragePooling1D layer
model2.add(GlobalAveragePooling1D())
#final dense layer
model2.add(Dense(1, activation='linear'))

model2.compile(loss=tf.keras.losses.mean_squared_error, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))
model2.summary()
history2 = model2.fit(X_train, Y_train, epochs=50, validation_data=(X_test,Y_test2), batch_size=64)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# Plotting training and validation loss
plt.figure(figsize=(10, 6))

# Training loss
plt.plot(history2.history['loss'], label='Training Loss', marker='o')

# Validation loss
plt.plot(history2.history['val_loss'], label='Validation Loss', marker='o')

# Adding labels and title
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Performance(dias) for input shape (1650,64,4), no interpolation')
plt.legend()

# Adding grid for better readability
plt.grid(True)

# Show plot
plt.show()

In [ ]:
predictions = model2.predict(X_test)
Y_pred2 = []
for i in range(len(X_test)):
  Y_pred2.append(predictions[i])
yf['predicted-dias'] = Y_pred2
yf['test-set-dias'] = Y_test2
print(yf['predicted-dias'].dtype)
yf

In [ ]:
#Calculating Mean of input data
hdf['mean1'] = hdf['instphase_channel1'].apply(lambda arr: np.mean(arr) if len(arr) > 0 else np.nan)
hdf['mean2'] = hdf['instphase_channel2'].apply(lambda arr: np.mean(arr) if len(arr) > 0 else np.nan)
hdf['mean3'] = hdf['instphase_channel3'].apply(lambda arr: np.mean(arr) if len(arr) > 0 else np.nan)
hdf['mean4'] = hdf['instphase_channel4'].apply(lambda arr: np.mean(arr) if len(arr) > 0 else np.nan)
hdf
# meaninput = hdf[['mean1','mean2','mean3','mean4']].to_numpy()
# mergedinput = df.to_numpy()
# mergedinput.shape

###########################################


#######################################################

#############################################################################################


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/blusim/mat_15.csv')
df.head(150)

In [ ]:
row_count_by_timestamp = df['Timestamp'][0].value_counts()
# Print the results
print("Number of rows for each timestamp:")
print((row_count_by_timestamp))

In [ ]:
rdf = pd.read_csv("/content/drive/MyDrive/blusim/mat_15.csv")
rdf['Timestamp'] = pd.to_datetime(rdf['Timestamp'], format='%H:%M:%S')
rdf['Timestamp'] = rdf['Timestamp'].dt.strftime('%H:%M:%S')
rdf = rdf.groupby('Timestamp').agg(lambda x: x.tolist()).reset_index()

# Function to append next values to the first list
def append_next_values(lst, next_values):
    return lst[:1] + next_values

# Iterate over the DataFrame and append next values to the first list
for i in range(len(rdf) - 1):
    next_values = rdf.iloc[i + 1, 1:].tolist()
    for col in rdf.columns[1:]:
        rdf.at[i, col] = append_next_values(rdf.at[i, col], next_values)

# Pad the lists to a length of 100
for col in rdf.columns[1:]:
    rdf[col] = rdf[col].apply(lambda x: x + [None] * (100 - len(x)))

# Print or use the modified DataFrame as needed
rdf
